In [ ]:
from google.colab import drive
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from skimage.feature import hog


In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile

In [ ]:
zip_path = '/content/drive/MyDrive/archive (5).zip'
extract_path = '/content/dataset'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
# Paths to the dataset
train_csv_path = os.path.join(extract_path, 'train.csv')
val_csv_path = os.path.join(extract_path, 'val.csv')
train_images_path = os.path.join(extract_path, 'train')
val_images_path = os.path.join(extract_path, 'val')

In [ ]:
# Load the CSV files
train_labels = pd.read_csv(train_csv_path)
val_labels = pd.read_csv(val_csv_path)

In [ ]:
def load_images_from_folder(folder):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        # Check if the item is a directory (and not the classname.txt file)
        if os.path.isdir(label_folder) and label != 'classname.txt':
            for filename in os.listdir(label_folder):
                img_path = os.path.join(label_folder, filename)
                img = cv2.imread(img_path, cv2.IMREAD_COLOR)
                if img is not None:
                    images.append(img)
                    labels.append(label)
    return images, labels

In [ ]:
# Load training images
train_images, train_image_labels = load_images_from_folder(train_images_path)

In [ ]:
# Load validation images
val_images, val_image_labels = load_images_from_folder(val_images_path)

In [ ]:

# Convert labels to numeric format
label_dict = {'cat': 0, 'dog': 1}
train_image_labels = [label_dict[label] for label in train_image_labels]
val_image_labels = [label_dict[label] for label in val_image_labels]

In [ ]:
# Function to preprocess images
def preprocess_images(images):
    processed_images = []
    for img in images:
        img_resized = cv2.resize(img, (128, 128))  # Resize to 128x128
        img_normalized = img_resized / 255.0  # Normalize to [0, 1]
        processed_images.append(img_normalized)
    return np.array(processed_images)

In [ ]:
# Preprocess the images
train_images_processed = preprocess_images(train_images)
val_images_processed = preprocess_images(val_images)


In [ ]:

# Function to extract HOG features
def extract_hog_features(images):
    hog_features = []
    for img in images:
        feature = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), multichannel=True)
        hog_features.append(feature)
    return np.array(hog_features)


In [ ]:

# Extract HOG features
train_features = extract_hog_features(train_images_processed)
val_features = extract_hog_features(val_images_processed)


<ipython-input-52-fdeec7b14f06>:5: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  feature = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), multichannel=True)


In [ ]:

# Scale the features
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
val_features_scaled = scaler.transform(val_features)


In [ ]:

# Train the SVM
svm = SVC(kernel='linear')
svm.fit(train_features_scaled, train_image_labels)


SVC(kernel='linear')

In [ ]:

# Predict on validation set
val_predictions = svm.predict(val_features_scaled)


In [ ]:

# Evaluate the model
accuracy = accuracy_score(val_image_labels, val_predictions)
report = classification_report(val_image_labels, val_predictions, target_names=['cat', 'dog'])



In [ ]:
print(f'Validation Accuracy: {accuracy}')
print(report)

Validation Accuracy: 0.6285714285714286
              precision    recall  f1-score   support

         cat       0.42      0.21      0.28        24
         dog       0.67      0.85      0.75        46

    accuracy                           0.63        70
   macro avg       0.54      0.53      0.51        70
weighted avg       0.58      0.63      0.59        70

